In [29]:
# -*- coding: utf-8 -*-

# Bayesian Personalized Ranking from Implicit Feedback
"""
data understanding
train.txt size      4*80000  
test.txt size       4*20000 
user                943
item                1682
 ____________________________________
|userid | itemid | preference | time |
|-------|--------|------------|------|
|_______|________|____________|______|

"""

import os
import numpy as np
import random
#from sklearn.metrics import roc_auc_score

unum = 943
inum = 1682

# Parameters Preset
alpha = 0.01
lamda_W = 0.01
lamda_H_pos = 0.01
lamda_H_neg = -0.01

In [30]:
""" Data load in, 943 users and 1682 items """

f1 = open("train.txt")
f2 = open("test.txt")

Rm = np.zeros((943, 1682))
Tm = np.zeros((943, 1682))
Tm_hat = np.zeros((943, 1682)) #retry
Pm = np.random.randn(943, 10) * 0.5
Qm = np.random.randn(1683, 10) * 0.5

f1.seek(0)
for it in range(80000):
    record = f1.readline().split('\t')
    u = int(record[0]) - 1
    i = int(record[1]) - 1
    Rm[u][i] = int(record[2])

f2.seek(0)
for it in range(20000):
    record = f2.readline().split('\t')
    u = int(record[0]) - 1
    i = int(record[1]) - 1
    Tm[u][i] = int(record[2]) - 1

In [31]:
def sigmond(x):
    return 1/(1 + np.exp(-x))

In [32]:
def train():
    """
    patial lnsigmaXuij patial theta
    exp(-x)/(1+exp(-x) * patial x patial theta)
    """
    global Pm
    global Qm
    for u in range(unum):
        for i in range(inum):
            if(Rm[u][i] >= 2):
                j = random.randint(0, inum-1)
                while Rm[u][j]>=2:
                    j = random.randint(0, inum-1)
                Xuij_hat = np.dot(Pm[u], Qm[i].T) - np.dot(Pm[u], Qm[j].T)
                sig_X_hat = sigmond(Xuij_hat)
                t_pu = Pm[u]
                t_qi = Qm[i]
                t_qj = Qm[j]
                Pm[u] = t_pu + alpha*((1-sig_X_hat) *(t_qi - t_qj) - lamda_W * t_pu)
                Qm[i] = t_qi + alpha*((1-sig_X_hat) *t_pu - lamda_H_pos * t_qi)
                Qm[j] = t_qj + alpha*((1-sig_X_hat) *(-t_pu) - lamda_H_neg * t_qj)

In [33]:
def test():
    ans = np.zeros(1586126)
    global Tm_hat
    for u in range(unum):
        for i in range(inum):
            Tm_hat[u][i] = np.dot(Pm[u],Qm[i].T)
            if Tm_hat[u][i] > 3:
                ans[u*1682+i] = 1
            else:
                ans[u*1682+i] = 0
    return ans

In [34]:
def auc_mine(Train, Test, Test_hat):
    up = 0
    under = 0
    Sum = 0
    for u in xrange(unum):
        for i in xrange(inum):
            if(Train[u][i] != 0):
                for j in xrange(inum):
                    if(Test[u][j] == 0 and Test[u][j] == 0):
                        under += 1
                        if(Test_hat[u][i]>Test_hat[u][j]):
                            up += 1
        Sum += float(up)/under
    return Sum/unum

In [38]:
for i in xrange(200):
    print "iter", i
    train()
loss = auc_mine(Rm, Tm, Tm_hat)

print "iter", i, loss

iter 0
iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9
iter 10
iter 11
iter 12
iter 13
iter 14
iter 15
iter 16
iter 17
iter 18
iter 19
iter 20
iter 21
iter 22
iter 23
iter 24
iter 25
iter 26
iter 27
iter 28
iter 29
iter 30
iter 31
iter 32
iter 33
iter 34
iter 35
iter 36
iter 37
iter 38
iter 39
iter 40
iter 41
iter 42
iter 43
iter 44
iter 45
iter 46
iter 47
iter 48
iter 49
iter 50
iter 51
iter 52
iter 53
iter 54
iter 55
iter 56
iter 57
iter 58
iter 59
iter 60
iter 61
iter 62
iter 63
iter 64
iter 65
iter 66
iter 67
iter 68
iter 69
iter 70
iter 71
iter 72
iter 73
iter 74
iter 75
iter 76
iter 77
iter 78
iter 79
iter 80
iter 81
iter 82
iter 83
iter 84
iter 85
iter 86
iter 87
iter 88
iter 89
iter 90
iter 91
iter 92
iter 93
iter 94
iter 95
iter 96
iter 97
iter 98
iter 99
iter 100
iter 101
iter 102
iter 103
iter 104
iter 105
iter 106
iter 107
iter 108
iter 109
iter 110
iter 111
iter 112
iter 113
iter 114
iter 115
iter 116
iter 117
iter 118
iter 119
iter 120
iter 121
iter 122
ite

# answer 

iter 400 0.884735054222